### Numerical features performed much better than categorical ones (alone)
### Combining both produced almost the same as just numerical, or slightly better
### Need a way to cluster head of household category 

### TODO FEATURE ENGINEERING

### Adding income levels based on regions (sonia) average income level for each region in the training set
### for each household, add # of sd's they are away from the average income level in their region

### Groupings for grade: none, elementary, high school, bachelors, masters, program/trade school (sonia)

### Percentage of family employed (# employed / total family) + interaction between the two terms (sonia)

### (hans) the rest

### Total number of electronics
### Total number of motorized vehicles
### housing/water expenditure x toilet facility interaction
### Clustering types of jobs

In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import normalize

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def select_features_from_lasso(X, y, alpha):
    # fit lasso model and pass to select from model
    Xtrain =normalize(X)
    lasso = Lasso(alpha).fit(Xtrain, y)
    model = SelectFromModel(lasso, prefit=True)

    # new features
    X_new = model.transform(X)
    return X.columns[model.get_support()]

In [3]:
def backward_stepwise(X, y):
    model = LinearRegression()

    vars_remaining = []
    vars_in_model = list(X.columns)
    
    last_error = cross_val_score(model, X[vars_in_model], y, cv=10, scoring="neg_mean_squared_error").mean()
    
    for _ in range(len(vars_in_model)):
        scores = []
        for var in vars_in_model:
            candidate_vars = copy.copy(vars_in_model)
            candidate_vars.remove(var)
            Xtrain = X[candidate_vars]
            scores.append(
                cross_val_score(model, normalize(Xtrain), y, cv=10, scoring="neg_mean_squared_error").mean()
            )
        i = np.argmax(scores)
        if scores[i] <= last_error:
            break
        else:
            last_error = scores[i]
            del vars_in_model[i]
            
    return vars_in_model

In [4]:
def forward_stepwise(X, y):
    model = LinearRegression()

    vars_remaining = list(X.columns)
    vars_in_model = []
    last_error = -np.inf

    for _ in range(len(vars_remaining)):
        scores = []
        for var in vars_remaining:
            candidate_vars = vars_in_model + [var]
            Xtrain = X[candidate_vars]
            scores.append(
                cross_val_score(model, normalize(Xtrain), y, cv=10, scoring="neg_mean_squared_error").mean()
            )
        i = np.argmax(scores)
        if scores[i] <= last_error:
            break
        else:
            last_error = scores[i]
            var_to_add = vars_remaining[i]
            vars_in_model.append(var_to_add)
            del vars_remaining[i]

    return vars_in_model

In [5]:
def get_categorical_numeric(df):
    numerics = []
    categorical = []

    for col in df:
        if((df[col].dtype == np.float64 or df[col].dtype == np.int64) and col != 'Unnamed: 0'):
            numerics.append(col)

        else:
            categorical.append(col)

    categorical_df = df[categorical]
    numeric_df = df[numerics]
    
    return categorical_df, numeric_df

In [6]:
def remove_categorical(X, feat, columns):
    for col in columns:
        if col in X:
            keep = []

            for f in feat:
                if col + '_' in f:
                    keep.append(f.replace(col + '_', ''))

            X[col][~(X[col].isin(keep))] = 'other'

In [7]:
def get_income_zscores(df):
    regions = pd.DataFrame(df[['Total Household Income', 'Region']]
        .groupby('Region').agg([np.mean, np.std]).to_records())
    regions.columns = ['Region', 'Average Household Income', 'Standard Deviation']

    z_scores = []

    for income, region in zip(df['Total Household Income'], df['Region']):
        idx = list(regions['Region']).index(region)
        avg = list(regions['Average Household Income'])[idx]
        sd = list(regions['Standard Deviation'])[idx]

        z_scores.append((income - avg) / sd)

    return z_scores

In [8]:
""" 
 use lasso regression to select columns for the model
 try a range of alpha parameters on the lasso model
 mode can be just categorical, numerical, or both
"""
def train_lasso(alphas, mode, df):
    mse = []
    y = df[['Total Household Income']] / 1e4
    df = df.drop(['Total Household Income', 'Unnamed: 0'], axis=1)
    
    for alpha in alphas:
        categorical_df, numeric_df = get_categorical_numeric(df)
        reg = LinearRegression()
        
        # only used categorical features
        # values not selected by lasso will be set to other
        if mode == "categorical":
            tmp = pd.get_dummies(categorical_df, columns = categorical_df.columns)
            new_feat = select_features_from_lasso(tmp, y, alpha)
            remove_categorical(categorical_df, new_feat, categorical_df.columns)
            categorical_df = pd.get_dummies(categorical_df, columns = categorical_df.columns)
            
            print ("Using alpha of {0} {1} columns were selected".format(alpha, len(new_feat)))
            print
            print (', '.join(new_feat))
            print 
            scores = cross_val_score(reg, categorical_df, y, cv=5, scoring='neg_mean_squared_error')
           
        # only use numerical features, and normalize the data
        elif mode == "numerical":
            new_feat = select_features_from_lasso(numeric_df, y, alpha)
            Xnum = preprocessing.scale(numeric_df[new_feat])
            
            print ("Using alpha of {0} {1} columns were selected".format(alpha, len(new_feat)))
            print 
            print (', '.join(new_feat))
            print
            
            scores = cross_val_score(reg, Xnum, y, cv=5, scoring='neg_mean_squared_error')
        
        # use both features
        else:
            new_feat_cat = select_features_from_lasso(pd.get_dummies(categorical_df, categorical_df.columns), 
                y, alpha)
            remove_categorical(categorical_df, new_feat_cat, categorical_df.columns)
            categorical_df = pd.get_dummies(categorical_df, columns = categorical_df.columns)
            
            new_feat_num = select_features_from_lasso(numeric_df, y, alpha)
            standardized = preprocessing.scale(numeric_df[new_feat_num])
            
            print ("Using alpha of {0} {1} columns were selected".format(alpha, len(categorical_df.columns) + 
                standardized.shape[1]))
            print
            print (', '.join(sorted(new_feat_num + new_feat_cat)))
            print
            
            total = np.concatenate((standardized, categorical_df.as_matrix()), axis=1)
            scores = cross_val_score(reg, total, y, cv=5, scoring='neg_mean_squared_error')

        print ("Alpha: {0}, MSE {1}".format(alpha, np.mean(scores)))
        print 

        mse.append(np.mean(scores))
        
    return mse

In [9]:
df = pd.read_csv('train.csv').dropna()

### Predict household income from just numerical features

### Use average household income to train regression model and produce a baseline MSE
### Very high baseline?

In [10]:
reg = LinearRegression()
scores = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error')
print ("Baseline MSE is {0}".format(np.mean(scores)))

NameError: name 'X' is not defined

### Lasso  regression to select numerical features
### Normalize features first? (Increased MSE ...)
### Alpha = 1e-5 selected 28 columns and produced the best MSE

In [ ]:
mse = train_lasso([1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1], "numerical", df)

In [ ]:
plt.plot(mse)
plt.xlabel('Alpha')
labels = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
plt.xticks(range(len(labels)), labels, rotation='vertical')
plt.ylabel('MSE')
plt.title('Effect of Alpha Parameter on MSE')
plt.show()

### Predict household income from just categorical features

### 500 columns after one-hot encoding and including all variables
### Household head occupation has 370 unique values (can probably be reduced)

### Alpha = 0.001 produced the best MSE and selected 107 columns
### For categorical variables, if the variable is chosen by lasso keep it, otherwise
### set it to an "other". Use the new dataframe in a linear regression model to
### get the mse

In [ ]:
mse = train_lasso([1e-3, 1e-2, 0.1, 1],'categorical',df)

In [ ]:
plt.plot(mse)
plt.xlabel('Alpha')
labels = [1e-3, 1e-2, 0.1, 1]
plt.xticks(range(len(labels)), labels, rotation='vertical')
plt.ylabel('MSE')
plt.title('Effect of Alpha Parameter on MSE')
plt.show()

### Use all the features to compute the MSE

In [ ]:
mse = train_lasso([1e-2, 0.1, 0.5, 1], 'both', df)

In [ ]:
plt.plot(mse)
plt.xlabel('Alpha')
labels = [1e-2, 0.1, 0.5, 1]
plt.xticks(range(len(labels)), labels, rotation='vertical')
plt.ylabel('MSE')
plt.title('Effect of Alpha Parameter on MSE')
plt.show()

### Engineer features to improve performance
### Add z-score for household income by region

In [ ]:
df = pd.read_csv('train.csv').dropna()
df['Income Z-Score'] = get_income_zscores(df)

In [ ]:
df.head()

In [ ]:
mse = train_lasso([1e-2, 0.1, 0.5, 1], 'both', df)